<a href="https://colab.research.google.com/github/MShiloni22/DDBMS_Project_A/blob/master/DDBMS_Project_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [2]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7b06dfe7d59b92c2eb66f254e1da77d61fcb366bec5a2365d4f9876bcfcde3f2
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
import os
import findspark
import datetime as dt

findspark.init()
from pyspark.sql import SparkSession
def init_spark(app_name: str):
 spark = SparkSession.builder.appName(app_name).getOrCreate()
 sc = spark.sparkContext
 return spark, sc
spark, sc = init_spark('demo')
sc

<SparkContext master=local[*] appName=demo>

In [57]:
# working on queries.csv extraction
from pyspark.sql import SparkSession,Row, Column
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
queries_file = '/content/drive/MyDrive/Colab Files/DDBMS//queries.csv'
df = spark.read.csv(queries_file, header='True', inferSchema='True')

print("Loaded queries successfully")

# doing the same process for all columns
column_names = ["genres", "lang", "actors", "director", "cities", "country", 
                "from_realese_date", "production_company"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "'[]", "")\
                .alias(temp_name_1))\
  .drop(name)

  # converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Cleaned and seperated to arrays")

# creating columns for each element in each column contains an array
for name in column_names:
  name_size = name + "_size"
  # get max array's size in the column
  df = df.withColumn(name_size, F.size(F.col(name)))
  df_col_max = df.agg({name_size: 'max'})
  max_col_size = df_col_max.collect()[0][0]

  for i in range(max_col_size):
    df = df.withColumn(name + "_" + str(i), F.col(name)[i])

  df = df.drop(name)
  df = df.drop(name_size)

df.show()


Loaded queries successfully
Cleaned and seperated to arrays
+-------+---------+----------------+----------------+-------+---------+------+------------------+----------------+--------------------+---------------+----------------+------------+----------+--------------+----------+--------+-----------+----------------+---------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|user_id| genres_0|        genres_1|        genres_2| lang_0|   lang_1|lang_2|          actors_0|        actors_1|            actors_2|       actors_3|      director_0|  director_1|director_2|      cities_0|  cities_1|cities_2|  country_0|       country_1|country_2|from_realese_date_0|from_realese_date_1|from_realese_date_2|production_company_0|production_company_1|production_company_2|production_company_3|production_company_4|
+-------+---------+----------------+----------------+-------+---------+-----

In [58]:
# working on credits.csv extraction
import re
credits_file = '/content/drive/MyDrive/Colab Files/DDBMS//credits.csv'
df = spark.read.csv(credits_file, header='True', inferSchema='True')

# load the data as you did before,
# just now change the delimiter to get evreything together
credits = spark.read.format("csv")\
.option("delimiter", "\t")\
.option("header","true")\
.option("inferSchema", "true")\
.load("drive/MyDrive/Colab Files/DDBMS//credits.csv")
prog = re.compile('\\[(.*?)\\]')
second_match = F.udf(lambda x: prog.findall(x)[1])
id_extract = F.udf(lambda x: x.split(",")[-1])
credits = credits\
.withColumn("id", id_extract("cast,crew,id"))\
.withColumn("cast", F.regexp_extract(F.col("cast,crew,id"), '\\[(.*?)\\]', 0
))\
.withColumn("crew", F.concat(F.lit("["),second_match("cast,crew,id"), F.lit(
"]")))\
.select("cast", "crew", "id")
df = credits
# df.printSchema()
print("Loaded credits successfully")

# doing the same process for all columns
column_names = ["cast", "crew"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # # converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Cleaned and seperated to arrays")

# for cast column - udf for extracting actors' names only from cast json string
actors_udf = F.udf(lambda arr: [arr[i][7:] for i in range(len(arr)) if i % 8 == 5])
df = df.withColumn('actors', actors_udf(F.col("cast")))\
  .drop("cast")

# for crew column - udf for extracting directors'' names only from cast json string
directors_udf = F.udf(lambda arr: [arr[i+1][7:] for i in range(len(arr))
 if arr[i] == " job: Director"])

df = df.withColumn('directors', directors_udf(F.col("crew")))\
  .drop("crew")

# converting arrays strings to arrays of strings
column_names = ["actors", "directors"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # # converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

for name in column_names:
  name_size = name + "_size"
  # get max array's size in the column
  df = df.withColumn(name_size, F.size(F.col(name)))
  df_col_max = df.agg({name_size: 'max'})
  max_col_size = df_col_max.collect()[0][0]

  for i in range(max_col_size):
    df = df.withColumn(name + "_" + str(i), F.col(name)[i])

  df = df.drop(name)
  df = df.drop(name_size)

df.show()

Loaded credits successfully
Cleaned and seperated to arrays
+-----+--------------------+--------------------+
|   id|              actors|           directors|
+-----+--------------------+--------------------+
|  862|[Tom Hanks,  Tim ...|     [John Lasseter]|
| 8844|[Robin Williams, ...|      [Joe Johnston]|
|15602|[Walter Matthau, ...|     [Howard Deutch]|
|31357|[Whitney Houston,...|   [Forest Whitaker]|
|11862|[Steve Martin,  D...|     [Charles Shyer]|
|  949|[Al Pacino,  Robe...|      [Michael Mann]|
|11860|[Harrison Ford,  ...|    [Sydney Pollack]|
|45325|[Jonathan Taylor ...|      [Peter Hewitt]|
| 9091|[Jean-Claude Van ...|       [Peter Hyams]|
|  710|[Pierce Brosnan, ...|   [Martin Campbell]|
| 9087|[Michael Douglas,...|        [Rob Reiner]|
|12110|[Leslie Nielsen, ...|        [Mel Brooks]|
|21032|[Kevin Bacon,  Bo...|       [Simon Wells]|
|10858|[Anthony Hopkins,...|      [Oliver Stone]|
| 1408|[Geena Davis,  Ma...|      [Renny Harlin]|
|  524|[Robert De Niro, ...|   [Martin S